<a href="https://colab.research.google.com/github/vatsal-kr/Recipe-KMST/blob/main/KMST_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet SPARQLWrapper
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=720f8b0ee0bc1e6958f0a2ee452d594b0d52ab28bf74bd5959a7f65f399a0902
  Stored in directory: /root/.cache/pip/wheels/e8/9d/fd/0a594a1bc8d493935d319ac40d64d79f5b4117ef4d37b1b678
Successfully built ws4py


In [ ]:
import anvil.server


In [ ]:
anvil.server.connect("server_WIONFUIB7XXXWRJEH27B4TD2-DERCMR7TIJFOJCY3")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import pandas as pd

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [ ]:
# get list of common ingredients

ing_qstr = """SELECT DISTINCT ?ing ?ingLabel
WHERE
{
    ?food wdt:P279 wd:Q2095;
        wdt:P527 ?ing.
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""
sparql.setQuery(ing_qstr)
sparql.setReturnFormat(JSON)
ings = sparql.query().convert()

In [ ]:
ings_df = pd.io.json.json_normalize(ings['results']['bindings'])
ings_df = ings_df[['ing.value', 'ingLabel.value']]
ings_df.columns = ['qid', 'name']
ings_df['qid'] = ings_df['qid'].str.split('/').str[-1]

<ipython-input-6-c4cd6b13ce46>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  ings_df = pd.io.json.json_normalize(ings['results']['bindings'])


In [ ]:
ings_dct = dict(zip(ings_df['name'], ings_df['qid']))

In [ ]:
''' get closest match of entered ingredient and available list of ingredients '''

' get closest match of entered ingredient and available list of ingredients '

In [ ]:
def wiki_id(name):
  if name not in ings_dct:
    for dct_name, qid in ings_dct.items():
      if (name in dct_name) or (dct_name in name):
        return qid
    return None
  else:
    return ings_dct[name]

In [ ]:
def form_query(required_ing, forbidden_ing):
  query = [
    'SELECT DISTINCT ?food ?ingredient ?foodLabel ?ingredientLabel',
    'WHERE',
    '{'
  ]

  temp_union_query = [f'{{?food wdt:P527 wd:{x}.}}' for x in required_ing]
  for e in temp_union_query:
    query.append(e)
    query.append('UNION')
  query.pop()

  if len(forbidden_ing)>0:
    minus_query = [f'MINUS {{ ?food wdt:P527 wd:{x}. }}' for x in forbidden_ing]

    query.extend(minus_query)

  query_end = [
      '?food wdt:P527 ?ingredient.',
      'SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }',
      '}',
  ]
  query.extend(query_end)
  return query

In [ ]:

def get_results(required_ing, forbidden_ing, return_query=False):
  required_ing = [wiki_id(x) for x in required_ing if wiki_id(x) is not None]
  forbidden_ing = [wiki_id(x) for x in forbidden_ing if wiki_id(x) is not None]
  
  formatted_query = '\n'.join(form_query(required_ing, forbidden_ing))

  sparql.setQuery(' '.join(form_query(required_ing, forbidden_ing)))
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  df = pd.json_normalize(results['results']['bindings'])
  df = df[['foodLabel.value', 'ingredientLabel.value']]
  df.columns = ['Food', 'Ingredients']
  df = df.sort_values(by=['Food', 'Ingredients'])
  df = df.sort_values(by=['Food', 'Ingredients']).reset_index(drop=True)

  return df, formatted_query

In [ ]:
@anvil.server.callable
def fetch_recipes(required_ingredients,prohibited_ingredients):
  required = required_ingredients.split()
  forbidden = prohibited_ingredients.split()

  results, query = get_results(required, forbidden)

  if results.empty:
    val = False
  else:
    val = True
  
  return results.to_markdown(), val


In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored

In [ ]:
# required = ['tomato', 'bread']
# forbidden = ['mayonnaise', 'onion']

# results, query = get_results(required, forbidden)

In [ ]:
results